In [1]:
import pandas as pd
from datetime import datetime
from pymongo import MongoClient



In [2]:
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

### Create operations

In [3]:
def add_record(db, collection_name, data, key_field):
    
    try:
        db[collection_name].insert_one(data)
        print(f"Added {collection_name[:-1]}: {data.get(key_field)}")
    except Exception as e:
        print(f"Error adding {collection_name[:-1]} {data.get(key_field)}: {e}")

### Adding Record

In [4]:
# --- New student ---
new_student = {
    "userId": "STU020",
    "email": "student17@example.com",
    "firstName": "Ada",
    "lastName": "Eze",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
        "bio": "Passionate learner exploring data science and analytics.",
        "avatar": "https://example.com/avatar16.png",
        "skills": ["Python", "SQL"]
    },
    "isActive": True
}

# --- New course ---
new_course = {
    "courseId": "CRS012",
    "title": "Data Visualization with Power BI",
    "description": "Learn to design and develop interactive dashboards with Power BI.",
    "instructorId": "INST001",
    "category": "Data Science",
    "level": "beginner",
    "duration": 40,
    "price": 99.99,
    "tags": ["data", "visualization", "business-intelligence"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now(),
    "isPublished": True
}

# --- New enrollment ---
new_enrollment = {
    "enrollmentId": "ENR019",
    "studentId": "STU016",
    "courseId": "CRS009",
    "enrolledAt": datetime.now(),
    "status": "active"
}

# --- New lesson ---
new_lesson = {
    "lessonId": "LES030",
    "courseId": "CRS009",
    "title": "Getting Started with Power BI",
    "content": "This lesson introduces the Power BI interface and its core features.",
    "order": 1,
    "resources": ["https://docs.microsoft.com/en-us/power-bi/"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now()
}

# --- Insert all at once ---
records = [
    ("users", new_student, "userId"),
    ("courses", new_course, "courseId"),
    ("enrollments", new_enrollment, "enrollmentId"),
    ("lessons", new_lesson, "lessonId")
]

for collection, data, key in records:
    add_record(db, collection, data, key)

Added user: STU020
Added course: CRS012
Added enrollment: ENR019
Added lesson: LES030


### Active Student

In [24]:
def get_active_students(db):
    try:
        active_students = db.Users.find(
            {"role": "student", "isActive": True},
            {"_id": 0, "userId": 1, "firstName": 1, "lastName": 1, "email": 1}
        )

        active_students_list = list(active_students)

        if active_students_list:
        
            for s in active_students_list:
                print(f"ID: {s['userId']}, Name: {s['firstName']} {s['lastName']}, Email: {s['email']}")
        else:
            print("No active students found.")

    except Exception as e:
        print("Error fetching active students:", e)

get_active_students(db)

ID: STU003, Name: Lori Hernandez, Email: robert66@example.net
ID: STU006, Name: Eric Mcfarland, Email: edavenport@example.net
ID: STU011, Name: Anthony Washington, Email: dpatterson@example.com
ID: STU013, Name: Timothy Duke, Email: flemingtracy@example.net
ID: STU015, Name: Fernando Bowman, Email: kreyes@example.net


### Getting Course with Instructor

In [23]:
def get_courses_with_instructor(db):
    try:
        course_with_instructor = db.Courses.aggregate([
            {
                "$lookup": {
                    "from": "users",
                    "localField": "instructorId",
                    "foreignField": "userId",
                    "as": "instructor"
                }
            },
            {"$unwind": "$instructor"},
            {
                "$project": {
                    "_id": 0,
                    "courseId": 1,
                    "title": 1,
                    "category": 1,
                    "level": 1,
                    "instructorFirstName": "$instructor.firstName",
                    "instructorLastName": "$instructor.lastName",
                    "instructorEmail": "$instructor.email"
                }
            }
        ])

        results = list(course_with_instructor)

        if results:
            print("Courses with Instructor Information:\n")
            for course in results:
                print(f"Course ID: {course['courseId']}")
                print(f"Title: {course['title']}")
                print(f"Category: {course['category']}")
                print(f"Level: {course['level']}")
                print(f"Instructor: {course['instructorFirstName']} {course['instructorLastName']} ({course['instructorEmail']})\n")
        else:
            print("No courses found.")

    except Exception as e:
        print("Error fetching courses with instructor info:", e)

get_courses_with_instructor(db)

No courses found.


## Courses in specific Category

In [22]:
category = "Data Science"

def get_courses_by_category(db, category):
    try:
        # Fetch courses in the given category
        courses_in_category = db.Courses.find(
            {"category": category},
            {"_id": 0, "courseId": 1, "title": 1, "level": 1}
        )

        courses_list = list(courses_in_category)

        if courses_list:
            print(f"Courses in category '{category}':\n")
            for course in courses_list:
                print(f"- {course['title']} ({course['level'].capitalize()}) — ID: {course['courseId']}")
        else:
            print(f"No courses found in category '{category}'.")

    except Exception as e:
        print(f"Error fetching courses in category '{category}': {e}")


get_courses_by_category(db, category)

Courses in category 'Data Science':

- Introduction to Data Science (Beginner) — ID: CRS002


## Find Student Enrolled in a Particular Course

In [21]:
def get_students_in_course(db, course_id):
    try:
        # Aggregate students enrolled in a specific course
        students_in_course = db.Enrollments.aggregate([
            {"$match": {"courseId": course_id}},
            {
                "$lookup": {
                    "from": "Users",
                    "localField": "studentId",
                    "foreignField": "userId",
                    "as": "student"
                }
            },
            {"$unwind": "$student"},
            {
                "$project": {
                    "_id": 0,
                    "userId": "$student.userId",
                    "firstName": "$student.firstName",
                    "lastName": "$student.lastName",
                    "email": "$student.email"
                }
            }
        ])

        students_list = list(students_in_course)

        if students_list:
            print(f"Students enrolled in course {course_id}:\n")
            for s in students_list:
                print(f"- {s['firstName']} {s['lastName']} ({s['email']}) — ID: {s['userId']}")
        else:
            print(f"No students found for course {course_id}.")

    except Exception as e:
        print(f"Error fetching students for course '{course_id}': {e}")

# Use Case
course_id = "CRS005"
get_students_in_course(db, course_id)

Students enrolled in course CRS005:

- Nancy Baldwin (farleykristen@example.org) — ID: STU005


## Search Course By Title

In [20]:
search_term = "Cloud Computing"

def search_courses_by_title(db, search_term):
    try:
        matched_courses = db.Courses.find(
            {"title": {"$regex": search_term, "$options": "i"}},
            {"_id": 0, "courseId": 1, "title": 1, "category": 1, "level": 1}
        )

        courses = list(matched_courses)

        if courses:
            print(f"Courses matching '{search_term}':\n")
            for course in courses:
                print(f"- {course['title']} ({course['level'].capitalize()} | {course['category']})")
        else:
            print(f"No courses found matching '{search_term}'.")
            
    except Exception as e:
        print(f"Error searching courses with term '{search_term}': {e}")

search_courses_by_title(db, search_term)

Courses matching 'Cloud Computing':

- Cloud Computing with AWS (Advanced | Cloud Computing)


## Update a Users Information

In [26]:
def update_user_profile(db, user_id, bio, avatar, skills):
    try:
        result = db.Users.update_one(
            {"userId": user_id},
            {
                "$set": {
                    "profile.bio": bio,
                    "profile.avatar": avatar,
                    "profile.skills": skills
                }
            }
        )

        if result.matched_count > 0:
            print(f"User {user_id} profile updated successfully.\n")

            # Show the updated user profile
            updated_user = db.users.find_one(
                {"userId": user_id},
                {"_id": 0, "userId": 1, "profile": 1}
            )
            print(updated_user)
        else:
            print(f"No user found with ID {user_id}")

    except Exception as e:
        print(f"Error updating user {user_id}: {e}")



update_user_profile(
    db,
    user_id="STU015",
    bio="Enthusiastic learner with interest in AI and Machine Learning.",
    avatar="https://example.com/new_avatar.png",
    skills=["Python", "SQL", "Machine Learning"]
)

User STU015 profile updated successfully.

None


## Mark course as published

In [33]:


def publish_course(db, course_id):
    try:
        result = db.Courses.update_one(
            {"courseId": course_id},
            {"$set": {"isPublished": True, "updatedAt": datetime.now()}}
        )

        if result.matched_count > 0:
            print(f"Course {course_id} marked as published successfully.\n")

            # Display updated course details
            updated_course = db.courses.find_one(
                {"courseId": course_id},
                {"_id": 0, "courseId": 1, "title": 1, "isPublished": 1, "updatedAt": 1}
            )
            
        else:
            print(f"No course found with ID {course_id}")

    except Exception as e:
        print(f"Error marking course {course_id} as published: {e}")



publish_course(db, "CRS004")

Course CRS004 marked as published successfully.



## Update Assignment

In [32]:

def update_submission_grade(db, submission_id, grade, feedback, status):
    try:
        result = db.Submissions.update_one(
            {"submissionId": submission_id},
            {
                "$set": {
                    "grade": grade,
                    "feedback": feedback,
                    "status": status,
                    "gradedAt": datetime.now()
                }
            }
        )

        if result.matched_count > 0:
            print(f"Updated grade and status for submission {submission_id} successfully.\n")

            # Display the updated submission
            updated_submission = db.submissions.find_one(
                {"submissionId": submission_id},
                {"_id": 0, "submissionId": 1, "grade": 1, "feedback": 1, "status": 1, "gradedAt": 1}
            )
            
        else:
            print(f"No submission found with ID {submission_id}")

    except Exception as e:
        print(f"Error updating grade for submission {submission_id}: {e}")


update_submission_grade(db, "SUB010", 85, "Great improvement!", "graded")

Updated grade and status for submission SUB010 successfully.



## Adding Tags

In [36]:
def update_course_tags(db, course_id, tags):
    try:
        result = db.Courses.update_one(
            {"courseId": course_id},
            {"$addToSet": {"tags": {"$each": tags}}}
        )

        if result.matched_count > 0:
            print(f"Tags updated for course {course_id} successfully.\n")

            # Display the updated course tags
            updated_course = db.courses.find_one(
                {"courseId": course_id},
                {"_id": 0, "courseId": 1, "title": 1, "tags": 1}
            )
            
        else:
            print(f" No course found with ID {course_id}")

    except Exception as e:
        print(f"Error updating tags for course {course_id}: {e}")


update_course_tags(db, "CRS005", ["AI", "Data Science", "Python"])

Tags updated for course CRS005 successfully.



## Delet a user

In [ ]:
from datetime import datetime

def deactivate_user(db, user_id):
    try:
        result = db.Users.update_one(
            {"userId": user_id},
            {"$set": {"isActive": False, "updatedAt": datetime.now()}}
        )

        if result.matched_count > 0:
            print(f" User {user_id} has been deactivated successfully.")
        else:
            print(f"No user found with ID {user_id}.")

    except Exception as e:
        print(f" Error deactivating user {user_id}: {e}")


deactivate_user(db, "STU006")

 User STU006 has been deactivated successfully.


## Delete enrollment

In [40]:
def delete_enrollment(db, enrollment_id):
    try:
        result = db.Enrollments.delete_one({"enrollmentId": enrollment_id})
        
        if result.deleted_count > 0:
            print(f"Enrollment {enrollment_id} deleted successfully.")
        else:
            print(f" No enrollment found with ID {enrollment_id}.")

    except Exception as e:
        print(f"Error deleting enrollment {enrollment_id}: {e}")


delete_enrollment(db, "ENR005")

Enrollment ENR005 deleted successfully.


## Deleting Lesson

In [41]:
def delete_lesson(db, lesson_id):
    try:
        delete_result = db.Lessons.delete_one({"lessonId": lesson_id})
        print(f"Deleted {delete_result.deleted_count} lesson(s) with ID {lesson_id}")
    except Exception as e:
        print(f"Error deleting lesson {lesson_id}: {e}")


delete_lesson(db, "LES008")


Deleted 1 lesson(s) with ID LES008
